<a href="https://colab.research.google.com/github/Annieng184/Annieng184/blob/main/%5BProject%5D_Tiki_Web_Scraper_July_5th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 57.5 MB/s 
     |████████████████████████████████| 358 kB 30.5 MB/s 
     |████████████████████████████████| 4.1 MB 41.8 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 58 kB 4.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://d

In [2]:
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [30]:
DRIVER = None

In [31]:
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

In [32]:
def close_driver():
    global DRIVER
    if not DRIVER is None:
        print("Quitting driver...")
        DRIVER.quit()
        print("Done!")

    DRIVER = None

In [33]:
close_driver()
initialize_driver()

Initiating driver...
Finished!


In [34]:
DRIVER

<selenium.webdriver.chrome.webdriver.WebDriver (session="9982401fa81d3ea9f99bb022f6ade102")>

**PART 1: Input URL and find number of products within that site**

In [44]:
DRIVER.get('https://tiki.vn/laptop-may-vi-tinh-linh-kien/c1846') #input URL

In [45]:
products = DRIVER.find_elements(By.CLASS_NAME, 'product-item') #count elements
len(products)

48

In [51]:
each_product = products[1] #get 1 element out for example

In [40]:
each_product.get_attribute

<bound method WebElement.get_attribute of <selenium.webdriver.remote.webelement.WebElement (session="9982401fa81d3ea9f99bb022f6ade102", element="490993ff-e5b9-41ae-a486-e6b794bd0dd9")>>

**PART 2: Find 4 elements: name, price, product_url, image**

In [52]:
#name
name = each_product.find_element(By.TAG_NAME, 'h3')
print(name.text)

Phần Mềm Diệt Virus BKAV Profressional Internet Security (1 năm) - Hàng Chính Hãng


In [14]:
#price
price = each_product.find_element(By.CLASS_NAME, 'price-discount__price')
print(price.text)

239.000 ₫


In [15]:
#get product url
url = each_product.get_attribute('href')
print(url)

https://tiki.vn/bo-kich-song-wifi-xiaomi-repeater-pro-hang-nhap-khau-p17967481.html?spid=42240501


In [16]:
#image
image = each_product.find_element(By.TAG_NAME, 'source')
print(image.get_attribute('srcset'))

https://salt.tikicdn.com/cache/200x200/ts/product/68/76/90/fb3b1d8cedf13c4cbfdfe35605b1f84d.jpg.webp


In [46]:
#create a function that aggregates the above information
def scrap_item(item):
  name = item.find_element(By.TAG_NAME, 'h3').text
  price = item.find_element(By.CLASS_NAME, 'price-discount__price').text
  url = item.get_attribute('href')
  image = item.find_element(By.TAG_NAME, 'source').get_attribute('srcset')

  info = {
      'name' : name,
      'price' : price,
      'url': url,
      'image' : image
  }
  return info

In [47]:
extracted_items = []


In [48]:
for item in products:
  extracted_items.append(scrap_item(item))

In [20]:
print(extracted_items)

[]


**PART 3: Save results as CSV file**

In [21]:
import pandas as pd

In [53]:
df = pd.DataFrame(extracted_items, columns=['name', 'price', 'url', 'image'])

In [54]:
df

,name,price,url,image
0,"Bộ lưu điện, bộ cấp điện liên tục UPS Prolink ...",1.290.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMUUgg...,https://salt.tikicdn.com/cache/200x200/ts/prod...
1,Phần Mềm Diệt Virus BKAV Profressional Interne...,205.000 ₫,https://tiki.vn/phan-mem-diet-virus-bkav-profr...,https://salt.tikicdn.com/cache/200x200/ts/prod...
2,Ổ cứng SSD 128GB Lexar NS100 2.5-Inch SATA III...,515.000 ₫,https://tiki.vn/o-cung-ssd-128gb-lexar-ns100-2...,https://salt.tikicdn.com/cache/200x200/media/c...
3,,,https://tiki.vn/man-hinh-may-tinh-xiaomi-mi-de...,https://salt.tikicdn.com/cache/200x200/ts/prod...
4,Hộp mực dành cho máy in canon Lbp 6030/ 6030w,135.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMZob-...,https://salt.tikicdn.com/cache/200x200/ts/prod...
5,Phần mềm Microsoft 365 Family English APAC EM ...,1.290.000 ₫,https://tiki.vn/phan-mem-microsoft-365-family-...,https://salt.tikicdn.com/cache/200x200/ts/prod...
6,Bộ Phát Wifi TP-Link Archer C54 Băng Tần Kép C...,429.000 ₫,https://tiki.vn/bo-phat-wifi-tp-link-archer-c5...,https://salt.tikicdn.com/cache/200x200/ts/prod...
7,,,https://tiki.vn/usb-64gb-cao-cap-thiet-ke-nho-...,https://salt.tikicdn.com/cache/200x200/ts/prod...
8,Màn Hình LG 29'' 21:9 UltraWide 29WN600-W Full...,5.889.000 ₫,https://tka.tiki.vn/pixel/pixel?data=djAwMfEyP...,https://salt.tikicdn.com/cache/200x200/ts/prod...
9,Bộ Kích Sóng Wifi Xiaomi Repeater Pro - Hàng N...,243.000 ₫,https://tiki.vn/bo-kich-song-wifi-xiaomi-repea...,https://salt.tikicdn.com/cache/200x200/ts/prod...


In [55]:
df.to_csv('tiki.scraped.csv', index = False)